In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# stdlib
import os
import time

# syft absolute
import syft as sy
from syft import autocache

sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(
    name="test-domain-1",
    dev_mode=True,
    in_memory_workers=True,
    reset=True,
    create_producer=True,
)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

# From multiple local files

In [ ]:
canada_dataset_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/ca%20-%20feb%202021.csv?raw=True"
canada_file = autocache(canada_dataset_url, "csv")
canada_file

In [ ]:
brazil_dataset_url = "https://github.com/OpenMined/datasets/blob/main/trade_flow/brazil%20-%20mar%202022.csv?raw=True"
brazil_file = autocache(brazil_dataset_url, "csv")
brazil_file

In [ ]:
data_ptr = domain_client.upload_files([canada_file, brazil_file])
data_ptr

In [ ]:
data_ptr.syft_action_data

In [ ]:
@sy.syft_function_single_use(trade_data=data_ptr)
def sum_trade_value_mil(trade_data):
    # third party
    import opendp.prelude as dp
    import pandas as pd

    dp.enable_features("contrib")

    aggregate = 0.0
    base_lap = dp.m.make_base_laplace(
        dp.atom_domain(T=float),
        dp.absolute_distance(T=float),
        scale=10.0,
    )

    noise = base_lap(aggregate)

    total = 0

    for data in trade_data:
        print("Loading file:", data.file_name)
        df = pd.read_csv(data.file_name, low_memory=False)
        total += df["Trade Value (US$)"].sum()

    return (float(total / 1_000_000), float(noise))

In [ ]:
sum_trade_value_mil(trade_data=data_ptr)

In [ ]:
for blob in data_ptr.syft_action_data:
    domain_client.api.services.blob_storage.delete(blob.syft_blob_storage_entry_id)

    time.sleep(1)

    if os.path.exists(blob.file_name):
        print(f"Trying to delete file: {blob.file_name}")
        os.unlink(blob.file_name)

# From a single file

In [ ]:
single_data_ptr = domain_client.upload_files(canada_file)
single_data_ptr

In [ ]:
sum_trade_value_mil(trade_data=single_data_ptr)

In [ ]:
for blob in single_data_ptr.syft_action_data:
    domain_client.api.services.blob_storage.delete(blob.syft_blob_storage_entry_id)

    time.sleep(1)

    if os.path.exists(blob.file_name):
        print(f"Trying to delete file: {blob.file_name}")
        os.unlink(blob.file_name)

# From a folder

In [ ]:
domain_client.upload_files("./path/to/folder")

#### Cleanup local domain server

In [ ]:
if node.node_type.value == "python":
    node.land()